You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

In [1]:
#The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:

!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

Defaulting to user installation because normal site-packages is not writeable
ERROR: Ignored the following versions that require a different python version: 0.10.0 Requires-Python >=3.10,<4.0; 0.11.0 Requires-Python >=3.10,<4.0; 0.11.1 Requires-Python >=3.10,<4.0; 0.11.2 Requires-Python >=3.10,<4.0; 0.14.0 Requires-Python >=3.10,<=3.13; 0.14.0rc0 Requires-Python >=3.10,<3.12; 0.14.0rc1 Requires-Python >=3.10,<=3.13; 0.14.1 Requires-Python >=3.10,<=3.13; 0.14.3 Requires-Python >=3.10,<=3.13; 0.14.4 Requires-Python >=3.10,<=3.13; 0.16.0 Requires-Python >=3.10,<=3.13; 0.16.1 Requires-Python >=3.10,<=3.13; 0.16.2 Requires-Python >=3.10,<=3.13; 0.16.3 Requires-Python >=3.10,<=3.13; 0.17.0 Requires-Python >=3.10,<=3.13; 0.17.0rc0 Requires-Python >=3.10,<=3.13; 0.17.0rc1 Requires-Python >=3.10,<=3.13; 0.17.0rc2 Requires-Python >=3.10,<=3.13; 0.19.0 Requires-Python >=3.10,<=3.13; 0.22.0 Requires-Python >=3.10,<=3.13; 0.22.2 Requires-Python >=3.10,<=3.13; 0.22.3 Requires-Python >=3.10,<=3.13; 0

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [8]:
from crewai import Agent, Task, Crew

TypeError: unsupported operand type(s) for |: 'type' and 'NoneType'

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [4]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

ModuleNotFoundError: No module named 'utils'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan on Artificial Intelligence

1. Introduction
- Definition of Artificial Intelligence
- Importance and relevance in today's world
- Brief overview of the latest trends and key players in the industry

2. Key Points
- Latest Trends:
  - Machine Learning and Deep Learning advancements
  - Natural Language Processing (NLP) developments
  - AI ethics and bias concerns
- Key Players:
  - Google, Microsoft, IBM, and Amazon leading the AI market
  - Startups making waves in specific AI niches
  - Academic

I now can edit this blog post to ensure it aligns with the organization's writing style and meets journalistic best practices.

Final Answer:
# The Rise of Artificial Intelligence: Trends, Players, and Impact

## Introduction

Artificial Intelligence, commonly referred to as AI, is the simulation of human intelligence processes by machines. It involves the development of algorithms that enable computers to perform tasks that typically require human intelligence, such as learning, reasoning, and problem-solving. In today's digital age, AI has become increasingly important due to its ability to analyze vast amounts of data efficiently and make predictions based on patterns and trends. The latest trends in AI include advancements in Machine Learning and Deep Learning, which have led to significant breakthroughs in areas such as image recognition, natural language processing, and autonomous driving.

Key players in the AI industry, such as Google, Microsoft, IBM, and Amazon, are leading th

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Rise of Artificial Intelligence: Trends, Players, and Impact

## Introduction

Artificial Intelligence, commonly referred to as AI, is the simulation of human intelligence processes by machines. It involves the development of algorithms that enable computers to perform tasks that typically require human intelligence, such as learning, reasoning, and problem-solving. In today's digital age, AI has become increasingly important due to its ability to analyze vast amounts of data efficiently and make predictions based on patterns and trends. The latest trends in AI include advancements in Machine Learning and Deep Learning, which have led to significant breakthroughs in areas such as image recognition, natural language processing, and autonomous driving.

Key players in the AI industry, such as Google, Microsoft, IBM, and Amazon, are leading the way in developing cutting-edge AI technologies and applications. These tech giants are continually investing in research and development to stay ahead of the competition and drive innovation in the field. In addition to established companies, startups are also making waves in specific AI niches, focusing on niche areas such as healthcare, finance, and cybersecurity. Academic institutions are also playing a crucial role in advancing AI research, collaborating with industry partners to push the boundaries of what is possible with artificial intelligence.

## Latest Trends and Noteworthy News

Machine Learning and Deep Learning are at the forefront of AI advancements, with algorithms becoming increasingly sophisticated and capable of handling complex tasks. Natural Language Processing (NLP) is another area of rapid development, with AI systems now able to understand and generate human language with a high degree of accuracy. However, with these advancements come concerns about AI ethics and bias, as algorithms may inadvertently perpetuate existing biases or make decisions that have harmful consequences.

Recent breakthroughs in AI technology have had a profound impact on various industries, from healthcare and finance to transportation and retail. AI-powered solutions are being used to streamline operations, improve customer experiences, and drive innovation in ways previously thought impossible. Looking ahead, the possibilities for AI are endless, with predictions ranging from fully autonomous vehicles to AI-powered virtual assistants that can perform tasks on behalf of their users.

## Target Audience Analysis

Professionals in the technology and data science fields will find value in staying up-to-date on the latest trends and developments in AI. Business owners looking to implement AI solutions in their organizations can benefit from understanding the key players in the industry and how AI can drive business growth. Students and researchers interested in artificial intelligence will find the article informative and engaging, providing insights into the future of AI technology. Finally, a general audience curious about AI advancements will gain a deeper understanding of how AI is shaping our world and what the future holds for this transformative technology.

## Call to Action

I encourage readers to stay informed about AI news and developments by following reputable sources such as Forbes AI articles and the MIT Technology Review on AI advancements. Additionally, I recommend attending AI conferences and events for networking and learning opportunities to stay ahead of the curve in this rapidly evolving field. Finally, I prompt the audience to engage in discussions on AI topics, sharing their insights and perspectives on the impact of AI on businesses and society as a whole.

In conclusion, Artificial Intelligence is a powerful force that is reshaping the way we live, work, and interact with technology. By exploring the latest trends, key players, and noteworthy news in the AI industry, readers can gain a deeper understanding of the impact of AI on various industries and the possibilities for the future. Stay informed, stay engaged, and embrace the potential of AI to drive innovation and transform our world.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).

In [14]:
topic = "PV subcontractors opportunities and risks in Chile"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on PV subcontractors opportunities and risks in Chile.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: 
Title: Exploring PV Subcontractors Opportunities and Risks in Chile

Outline:
I. Introduction
- Brief overview of the solar energy industry in Chile
- Importance of PV subcontractors in the renewable energy sector
- Thesis statement highlighting the opportunities and risks for PV subcontractors in Chile

II. Latest Trends
- Growth of the solar energy market in Chile
- Increasing demand for PV installations
- Emergence of new technologies and innovations in the sector


I now can give a great answer

Final Answer:
# Exploring PV Subcontractors Opportunities and Risks in Chile

The solar energy industry in Chile has been experiencing significant growth, with an increasing demand for PV installations. PV subcontractors play a crucial role in the renewable energy sector, providing essential services for the development and maintenance of solar projects. In this article, we will delve into the opportunities and risks that PV subcontractors face in Chile, highlighting the latest trends, key players, noteworthy news, and the target audience's interests and pain points.

Chile has witnessed a remarkable growth in the solar energy market, driven by favorable government policies and increasing environmental awareness. The demand for PV installations has been on the rise, leading to a surge in opportunities for subcontractors to collaborate with developers and investors. Moreover, the emergence of new technologies and innovations in the sector has opened up exc

In [15]:
Markdown(result)

# Exploring PV Subcontractors Opportunities and Risks in Chile

The solar energy industry in Chile has been experiencing significant growth, with an increasing demand for PV installations. PV subcontractors play a crucial role in the renewable energy sector, providing essential services for the development and maintenance of solar projects. In this article, we will delve into the opportunities and risks that PV subcontractors face in Chile, highlighting the latest trends, key players, noteworthy news, and the target audience's interests and pain points.

Chile has witnessed a remarkable growth in the solar energy market, driven by favorable government policies and increasing environmental awareness. The demand for PV installations has been on the rise, leading to a surge in opportunities for subcontractors to collaborate with developers and investors. Moreover, the emergence of new technologies and innovations in the sector has opened up exciting possibilities for PV subcontractors to enhance their services and expand their market presence.

Leading PV subcontractors in Chile have been actively involved in major solar projects, showcasing their expertise and capabilities in the renewable energy sector. Collaboration between international and local companies has been instrumental in driving innovation and efficiency in solar installations. Government initiatives supporting the growth of the solar industry have created a conducive environment for PV subcontractors to thrive and contribute to the country's sustainable energy goals.

Recent projects and investments in the solar energy sector have highlighted the potential for further expansion and development in Chile. Regulatory updates impacting PV subcontractors have brought about challenges in compliance and operations, requiring careful navigation and adaptation. Despite the challenges faced by subcontractors, potential solutions such as improved coordination and communication within the industry offer opportunities for growth and resilience.

Professionals in the renewable energy industry, investors looking to enter the Chilean solar market, and policy makers and regulators involved in the energy sector form the target audience for this article. Their interests lie in the potential for growth and investment opportunities in the solar energy sector, while their pain points revolve around regulatory uncertainty and market competition. Providing reliable information and guidance on navigating the PV subcontractor landscape in Chile is essential for addressing their needs and concerns.

Encourage readers to explore partnership opportunities with PV subcontractors in Chile, provide resources for further information on solar energy regulations and market trends, and invite feedback and questions from the audience to foster engagement.

In conclusion, the opportunities and risks for PV subcontractors in Chile present a dynamic landscape of growth and challenges, requiring informed decision-making and strategic partnerships. By staying informed and proactive in addressing industry trends and developments, PV subcontractors can navigate the evolving renewable energy sector in Chile with confidence and success.

This blog post aims to provide valuable insights and guidance for professionals in the renewable energy industry, investors, and policy makers, enabling them to make informed decisions and contribute to the sustainable development of the solar energy sector in Chile.

In [ ]:
topic = "GenAI for SCADA signal analysis and WhatsApp notifications with CrewAI."
result = crew.kickoff(inputs={"topic": topic})

: 

In [9]:
from crewai import Agent, Task, Crew

# Define your agents
researcher = Agent(
    role="Researcher",
    goal="Conduct thorough research and analysis on AI and AI agents",
    backstory="You're expert researcher, specialized in technology, software engineering, AI, and startups. You work as a freelancer and are currently researching for a new client.",
    allow_delegation=False,
)

writer = Agent(
    role="Senior Writer",
    goal="Create compelling content about AI and AI agents",
    backstory="You're a senior writer, specialized in technology, software engineering, AI, and startups. You work as a freelancer and are currently writing content for a new client.",
    allow_delegation=False,
)

# Define your tasks
task1 = Task(
 description="Conduct a comprehensive analysis of the latest advancements in AI in 2024. Identify key trends, breakthrough technologies, and potential industry impacts. Compile your findings in a detailed report.",
    expected_output="A comprehensive full report on the latest AI advancements in 2024, leave nothing out",
    agent=researcher,
    human_input=True
)

task2 = Task(
    description="Using the insights from the researcher's report, develop an engaging blog post that highlights the most significant AI advancements. Your post should be informative yet accessible, catering to a-savvy audience.",
    expected_output="A compelling 3 paragraphs blog post formatted as markdown about the latest AI advancements in 2024",
    agent=writer,
    human_input=True
)

# Instantiate your crew with a sequential process
crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    verbose=True,
    memory=True,
    planning=True  # Enable planning feature for the crew
)

# Get your crew to work!
result = crew.kickoff()

# Print the result
print("print:",result)

TypeError: unsupported operand type(s) for |: 'type' and 'NoneType'